In [1]:
    # ======================
    # 1. Instalar librerías
    # ======================
    # !pip install feedparser beautifulsoup4 requests pandas sentence-transformers scikit-learn openpyxl

In [15]:
# ======================
# 2. Importar librerías
# ======================
import feedparser
import requests
from bs4 import BeautifulSoup
from dateutil import parser
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from tqdm import tqdm
import unicodedata
import re
import numpy as np
import html


# ======================
# 3. Cargar archivos base
# ======================
medios_df = pd.read_excel("medios_colombia.xlsx")#Contiene la información de medios de comunicación
df_palabras = pd.read_excel("palabras_clave.xlsx")#Contiene las palabras clave para filtrar las noticias
palabras_clave = df_palabras['palabra_clave'].dropna().str.lower().tolist()
df_mpios = pd.read_excel("municipio.xlsx")#Contiene los departamentos y municipios según el DANE
df_contexto = pd.read_excel("contexto_territorio.xlsx")#Contiene palabras de contexto territorial para afinar la asignación de territorio a la noticia


# ============================
# 4. Funciones de recolección de noticias (rss o scraping)
# ============================
def extraer_rss(url):
    try:
        feed = feedparser.parse(url)
        noticias = []
        for entry in feed.entries:
            titulo = entry.title
            link = entry.link
            fecha = entry.get('published', '') or entry.get('updated', '')
            contenido_raw = entry.get('summary', '') or entry.get('description', '')
            contenido = limpiar_contenido_html(contenido_raw)
            noticias.append({
                'titulo': titulo,
                'fecha': datetime.now().isoformat(),
                'link': link,
                'contenido': contenido
            })
        return noticias
    except Exception as e:
        print(f"Error en RSS {url}: {e}")
        return []

def extraer_scraping(url, selector):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        elementos = soup.select(selector)
        noticias = []
        for e in elementos:
            texto_bruto = str(e)
            texto_limpio = limpiar_contenido_html(texto_bruto)
            if texto_limpio:
                noticias.append({
                    'titulo': texto_limpio[:100],  # puedes ajustar cómo se define el título
                    'fecha': datetime.now().isoformat(),
                    'link': url,
                    'contenido': texto_limpio
                })
        return noticias
    except Exception as e:
        print(f"Error en scraping {url}: {e}")
        return []


# =======================================
# 5. Filtrar noticias por palabras clave
# =======================================
def extraer_palabras_clave(texto, palabras_clave):
    texto = texto.lower()
    coincidencias = [palabra for palabra in palabras_clave if palabra in texto]
    return coincidencias if coincidencias else None


# =======================================
# 6. Eliminar duplicados por similitud
# =======================================
def eliminar_duplicados(lista_noticias, umbral=0.8):

    # Convertir lista de dicts a DataFrame
    df = pd.DataFrame(lista_noticias)

    # Concatenar título + contenido para tener mayor contexto
    textos = (df['titulo'].fillna('') + ' ' + df['contenido'].fillna('')).tolist()

    # Cargar modelo semántico
    modelo = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = modelo.encode(textos, convert_to_tensor=True)

    # Calcular similitud coseno
    similitudes = util.cos_sim(embeddings, embeddings)

    # Marcar duplicados (solo superior a la diagonal)
    duplicados = set()
    for i in range(len(similitudes)):
        for j in range(i + 1, len(similitudes)):
            if similitudes[i][j] > umbral:
                duplicados.add(j)

    # Filtrar no duplicados
    df_filtrado = df.drop(index=list(duplicados)).reset_index(drop=True)

    # Convertir de nuevo a lista de dicts
    return df_filtrado.to_dict(orient='records')


# =======================================
# 7. Asignar municipio y departamento (mejorado)
# =======================================
def asignar_ubicacion(noticia, df_mpios):
    contexto_territorio = df_contexto['contexto'].dropna().str.strip().str.lower().tolist()
    texto = normalizar_texto(noticia['titulo'] + " " + noticia['contenido'])

    # Revisar combinaciones más confiables: municipio seguido de departamento
    for _, row in df_mpios.iterrows():
        municipio = normalizar_texto(str(row['municipio']))
        departamento = normalizar_texto(str(row['departamento']))

        # Ejemplo: "mallama, nariño"
        patron_comb = r'\b' + re.escape(municipio) + r'\s*,\s*' + re.escape(departamento) + r'\b'
        if re.search(patron_comb, texto):
            noticia['municipio'] = row['municipio']
            noticia['departamento'] = row['departamento']
            noticia['cod_municipio'] = row['cod_municipio']
            noticia['cod_departamento'] = row['cod_departamento']
            return noticia

    # Buscar municipios con contexto
    for _, row in df_mpios.iterrows():
        municipio = normalizar_texto(str(row['municipio']))
        for contexto in contexto_territorio:
            patron = r'\b' + re.escape(contexto) + r'\s+' + re.escape(municipio) + r'\b'
            if re.search(patron, texto):
                noticia['municipio'] = row['municipio']
                noticia['departamento'] = row['departamento']
                noticia['cod_municipio'] = row['cod_municipio']
                noticia['cod_departamento'] = row['cod_departamento']
                return noticia

    # Buscar departamentos con contexto (solo si no se encontró municipio)
    for _, row in df_mpios.iterrows():
        departamento = normalizar_texto(str(row['departamento']))
        for contexto in contexto_territorio:
            patron = r'\b' + re.escape(contexto) + r'\s+' + re.escape(departamento) + r'\b'
            if re.search(patron, texto):
                noticia['municipio'] = None
                noticia['departamento'] = row['departamento']
                noticia['cod_municipio'] = None
                noticia['cod_departamento'] = row['cod_departamento']
                return noticia

    # Si no se encuentra nada
    noticia['municipio'] = None
    noticia['departamento'] = None
    noticia['cod_municipio'] = None
    noticia['cod_departamento'] = None
    return noticia


# =======================================
# 8. Estandariza la fecha
# =======================================
def estandarizar_fecha(fecha):
    """
    Convierte una fecha en múltiples formatos posibles al formato DD/MM/AAAA.
    Si no se puede interpretar, devuelve None.
    """
    try:
        fecha_dt = parser.parse(str(fecha), dayfirst=True, fuzzy=True)
        return fecha_dt.strftime('%d/%m/%Y')
    except (parser.ParserError, TypeError, ValueError):
        return None


# =======================================
# 9. limpiar campo "contenido"
# =======================================
def limpiar_contenido_html(texto):
    texto_limpio = BeautifulSoup(texto, "html.parser").get_text(separator=' ', strip=True)
    texto_limpio = html.unescape(texto_limpio)
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio).strip()
    return texto_limpio

    
# =======================================
# 10. Función principal
# =======================================
def ejecutar_pipeline(medios_df, palabras_clave, df_mpios):
    noticias_totales = []

    for _, row in tqdm(medios_df.iterrows(), total=len(medios_df), desc="Recolección"):
        tipo = row['tipo']
        url = row['url']
        medio = row['nombre_medio']
        selector = row.get('selector_scraping', None)

        if tipo.lower() == 'rss':
            noticias = extraer_rss(url)
        elif tipo.lower() == 'scrap' and pd.notna(selector):
            noticias = extraer_scraping(url, selector)
        else:
            continue

        for noticia in noticias:
            noticia['medio'] = medio
            palabras_encontradas = extraer_palabras_clave(noticia['titulo'] + " " + noticia['contenido'], palabras_clave)
            if palabras_encontradas:
                noticia['palabra_clave'] = ', '.join(palabras_encontradas)  
                noticias_totales.append(noticia)

    print(f"Noticias antes de eliminar duplicados: {len(noticias_totales)}")
    noticias_df = pd.DataFrame(noticias_totales)
    noticias_filtradas = eliminar_duplicados(noticias_df)
    print(f"Noticias después de eliminar duplicados: {len(noticias_filtradas)}")

    noticias_enriquecidas = [asignar_ubicacion(n, df_mpios) for n in tqdm(noticias_filtradas, desc="Asignando ubicación")]
    
    columnas_finales = ["medio", "titulo", "fecha", "link", "municipio", "cod_municipio",
                        "departamento", "cod_departamento", "palabra_clave", "contenido"]
    
    df_final = pd.DataFrame(noticias_enriquecidas)[columnas_finales]
    
    # Seleccionar los datos del día
    df_final['fecha'] = df_final['fecha'].apply(estandarizar_fecha)
    df_final['fecha'] = pd.to_datetime(df_final['fecha'], format='%d/%m/%Y', errors='coerce')
    hoy = pd.to_datetime(datetime.now().strftime('%d/%m/%Y'), format='%d/%m/%Y')
    df_hoy = df_final[df_final['fecha'] == hoy]

    return df_final


# =======================================
# 11 Ejecutar
# =======================================
noticias_final = ejecutar_pipeline(medios_df, palabras_clave, df_mpios)
#noticias_final.to_excel("noticias_resultado.xlsx", index=False)
#noticias_final.to_excel(f"noticias_resultado_{datetime.now().strftime('%Y-%m-%d')}.xlsx", index=False)
noticias_final.to_excel(f"resultados/noticias_resultado_{datetime.now().strftime('%Y-%m-%d')}.xlsx", index=False)

Buscando en El Tiempo: 100%|███████████████████████████████████████████████████████████| 22/22 [00:50<00:00,  2.31s/it]

✅ Archivo generado: noticias_eltiempo.xlsx
